In [ ]:
%load_ext autoreload
%autoreload 2
import math
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils
import torch.utils.data
import torch.nn as nn
import torchsummary
import torch.nn.functional as F
import numpy as np
from livenet.backend.core import Context
import random
import importlib
import onnx
import livenet
device = "cuda"
device = "cpu"
torch.set_default_device(device)



In [ ]:
# transform = transforms.Compose(
#     [transforms.ToTensor()
#         ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#      ])
# train=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=True,
#                                   download=True, transform=transform)
# test=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=False,
#                                   download=True, transform=transform)
#
#
# def get_whole_data(dataset):
#     loader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset))
#     data = next(iter(loader))
#     data[0] = data[0].to(device)
#     data[1] = data[1].to(device)
#     return data
#
# test_whole_data = get_whole_data(test)
# train_whole_data = get_whole_data(train)

# test_x, test_y = test_whole_data
# train_x, train_y = train_whole_data

test_x, test_y = livenet.datasets.get_cifar10_test()
train_x, train_y = livenet.datasets.get_cifar10_train()



In [ ]:


def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(0)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.context = Context(self)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.conv2 = nn.Conv2d(8, 8, 3,padding="same")
        self.conv3 = nn.Conv2d(16, 16, 3,padding="same")
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc = nn.Linear(8*7*7,10)
        self._alpha = 0.0001

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        # x = self.pool(x)
        # x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        x=self.fc(x)
        return x

    def internal_loss(self):
        loss = torch.tensor(0.)
        for param in self.parameters():
            if len(param.data.shape) > 1:
                loss += self._alpha * torch.sum(torch.abs(param)) / param.data.numel()
        return loss


network = Net()
torchsummary.summary(network, (3, 32, 32), device=device)


In [ ]:
batch_size = 256
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=len(train_x) // batch_size)
trainer.adaptive_lr = False


In [ ]:

trainer.step(5000)


In [ ]:

def calc_stats(network, data):
    with torch.no_grad():
        outputs = network(data[0])
    labels = data[1]

    total = len(labels)
    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    _loss = criterion(outputs, labels)
    _reg_loss = net.reg_loss_func()
    print(f"Loss {_loss:.2f} {_reg_loss:.2f} Accuracy {100 * correct / total:.1f}%")



optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
losses = []


In [ ]:
def test_func():
    return livenet.gen_utils.batch_iterator(*test_whole_data, batch_size=256, only_one_epoch=True)

def train_func():
    return livenet.gen_utils.batch_iterator(*train_whole_data, batch_size=256, only_one_epoch=True)
import time

net._alpha = 0.0001

def criterion(input, target):
    return nn.functional.cross_entropy(input, target) / math.log(2)

# optimizer = lib.optimizer.MyOptimizer(net.parameters(), lr=0.01)
t0 = time.time()
for epoch in range(50):
    print(f"{time.time() - t0:.3f} sec")
    t0 = time.time()

    running_loss = 0.0
    for i, data in enumerate(train_func(), 1):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss += net.reg_loss_func()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        n_observe = 4 * 195

        # print statistics
        running_loss += loss.item()
    print(f'[{epoch + 1}, {i:5d}] running loss: {running_loss / i:.2f}')
    running_loss = 0.0
    calc_stats(net, train_whole_data)
    calc_stats(net, test_whole_data)
    #lr schedule
    if True:
        observed = np.array(losses[-n_observe:])
        av1 = np.average(observed[:len(observed) // 2])
        av2 = np.average(observed[len(observed) // 2:])
        print(f"av1={av1:.4f} av2={av2:.4f}")
        slope, pvalue = livenet.stat_utils.get_slope_and_pvalue(losses[-n_observe:])
        print(f"slope={slope:.1e} pvalue={pvalue:.1e} lr={optimizer.param_groups[0]['lr']}")
        if slope >= 0.0:
            optimizer.param_groups[0]["lr"] /= 1.4
            print(f"reduced lr to {optimizer.param_groups[0]['lr']:.2e}")

print('Finished Training')


In [ ]:
importlib.reload(lib)
import matplotlib.pyplot as plt

it = iter(net.parameters())
conv1 = next(it).data.numpy()
next(it)
conv2 = next(it).data.numpy()
livenet.visual_utils.show_convs(conv1)



In [ ]:
importlib.reload(lib)
stat = lib.stat_utils.AccumStat()
for name, param in net.named_parameters():
    stat = lib.stat_utils.AccumStat(param.data)
    stat.plot(name)


